python原生的None和pandas, numpy中的numpy.NaN尽管在概念上都是用来标示空缺数据。但它们的行为在很多场合确有一些相当大的不同。这些不一致性曾经给我带来了不少麻烦，特此整理了一份详细的实验，比较None和NaN的行为异同。

**本文对应的Jupyter Notebook原文件可以在这里[下载](https://github.com/junjiecai/jupyter_labs/tree/master/cjj_labs/0006_None_vs_NaN),欢迎下载后亲自动手实验其中的代码。**

In [1]:
from numpy import NaN
from pandas import Series, DataFrame
import numpy as np

## 支持map?

In [2]:
s = Series([None, NaN, 'a'])
s

0    None
1     NaN
2       a
dtype: object

In [4]:
s.map({None:1,'a':'a'})

0    1
1    1
2    a
dtype: object

In [48]:
s.map({NaN:1,'a':'a'})

0    1
1    1
2    a
dtype: object

In [5]:
s.map({NaN:2,'None':1,'a':'a'})

0    2
1    2
2    a
dtype: object

In [6]:
s.map({'None':1,NaN:2,'a':'a'})

0    2
1    2
2    a
dtype: object

## 支持replace?

In [35]:
s = Series([None, NaN, 'a'])
s

0    None
1     NaN
2       a
dtype: object

In [38]:
s.replace([NaN],9)

0    9
1    9
2    a
dtype: object

In [37]:
s.replace([None],9)

0    9
1    9
2    a
dtype: object

会把None或者NaN一起替换

## 能作为key?

In [33]:
{None:1}

{None: 1}

In [34]:
{NaN:1}

{nan: 1}

In [55]:
{None:1, NaN:2}

{nan: 2, None: 1}

都可以

## 数据类型?
None是一个python特殊的数据类型， 但是NaN却是用一个特殊的float

In [2]:
type(None)

NoneType

In [3]:
type(NaN)

float

## 混入numpy.array的影响

如果数据中含有None,会导致整个array的类型变成object。

In [4]:
np.array([1, None]).dtype

dtype('O')

而np.NaN尽管会将原本用int类型就能保存的数据转型成float，但不会带来上面这个问题。

In [5]:
np.array([1, NaN]).dtype

dtype('float64')

注意numpy有不少函数可以自动处理NaN。

In [6]:
np.nansum([1,2,NaN])

3.0

但是None不能享受这些函数的便利，如果数据包含的None的话会报错

In [7]:
np.nansum([1,2,None])

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

## 混入对Series的影响

下面的结果估计大家能猜到

In [8]:
Series([1, NaN])

0    1.0
1    NaN
dtype: float64

下面的这个就很意外的吧

In [47]:
Series([1, None])

0    1.0
1    NaN
dtype: float64

pandas将None自动替换成了NaN！ 

一般情况下，Series都会让以数据中最抽象一般的数据类型为准作为整个Series的类型。 例如int是一种特殊的float,float是一种特殊的object，我们观察以下现象

In [48]:
Series([1,1])

0    1
1    1
dtype: int64

In [49]:
Series([1,1.1])

0    1.0
1    1.1
dtype: float64

In [51]:
Series([1,'a'])

0    1
1    a
dtype: object

上面三个例子里面，Series的数据结构都已数据中最抽象一般的那个数据的类型为准。然而这个例子里，

In [5]:
Series([1.0, None])

0    1.0
1    NaN
dtype: float64

却是Object类型的None被替换成了float类型的NaN。 这么设计可能是因为None无法参与numpy的大多数计算， 而pandas的底层又依赖于numpy，因此尽可能的希望Series是可计算的。 

不过如果本来Series就只能用object类型的话， Series不会做这样的转化工作。

In [6]:
Series(['a', None])

0       a
1    None
dtype: object

但是如果Series里面都是None的话却不会做这样的转化

In [6]:
Series([None,None])

0    None
1    None
dtype: object

最后， 如果Series中其他数据可以准化成float, astype(float)会把None转化成NaN

In [20]:
Series([None,'1']).astype(float)

0    NaN
1    1.0
dtype: float64

In [9]:
Series([True, False, None])

0     True
1    False
2     None
dtype: object

True, False完全可以用Float储存， 但是NaN和True, False一起存放的时候， 数据类型却是Object

In [19]:
Series([True, False]).astype(type(NaN))

0    1.0
1    0.0
dtype: float64

In [20]:
Series([True, False, NaN])

0     True
1    False
2      NaN
dtype: object

## 等值性判断

### 基本的等值性比较

In [97]:
None == None

True

In [98]:
NaN == NaN

False

In [99]:
None == NaN

False

### 在tuple中的情况

这个不奇怪

In [100]:
(1, None) == (1, None)

True

这个又和之前的结果不一样了

In [102]:
(1, NaN) == (1, NaN)

True

这个不意外

In [22]:
(1, None) == (1, NaN)

False

### 在numpy.array中的情况

In [146]:
np.array([1,None]) == np.array([1,None])

array([ True,  True], dtype=bool)

In [147]:
np.array([1,NaN]) == np.array([1,NaN])

array([ True, False], dtype=bool)

In [23]:
np.array([1,NaN]) == np.array([1,None])

array([ True, False], dtype=bool)

虽说它们和“基本的等值性判断”中的表现一致，但是大部分情况我们希望上面例子中， 左右两边的array被看做一致。这时可以用numpy.testing.assert_equal函数来处理。 注意这个函数的表现同assert， 不会返回True, False， 而是无反应或者raise Exception

In [26]:
np.testing.assert_equal(np.array([1,NaN]), np.array([1,NaN]))

它也可以处理两边都是None的情况

In [27]:
np.testing.assert_equal(np.array([1,None]), np.array([1,None]))

但是一边是None，一边是NaN时会被认为两边不一致, 导致AssertionError

In [24]:
np.testing.assert_equal(np.array([1,NaN]), np.array([1,None]))

AssertionError: 
Arrays are not equal

(mismatch 50.0%)
 x: array([  1.,  nan])
 y: array([1, None], dtype=object)

### 在Series中的情况

In [8]:
Series([None,'a']) == Series([None,'a'])

0    False
1     True
dtype: bool

上面这个例子和```None==None```的时候表现不一致

In [9]:
Series([NaN,'a']) == Series([NaN,'a'])

0    False
1     True
dtype: bool

In [28]:
Series([None,'a']) == Series([NaN,'a'])

0    False
1     True
dtype: bool

上面这两个和基本等值性比较的情况一致

和array类似，Series也有专门的函数equals用于判断两边的Series是否整体相等

In [29]:
Series([None,'a']).equals(Series([NaN,'a']))

True

In [30]:
Series([None,'a']).equals(Series([None,'a']))

True

In [31]:
Series([NaN,'a']).equals(Series([NaN,'a']))

True

比numpy.testing.assert_equals更智能些， 三种情况下都能恰当的处理

### 在DataFrame merge中的表现

两边的None会被判为相同

In [108]:
a = DataFrame({'A':[None,'a']})
b = DataFrame({'A':[None,'a']})
a.merge(b,on='A', how = 'outer')

,A
0,None
1,a


两边的NaN会被判为相同

In [110]:
a = DataFrame({'A':[NaN,'a']})
b = DataFrame({'A':[NaN,'a']})
a.merge(b,on='A', how = 'outer')

,A
0,NaN
1,a


In [32]:
a = DataFrame({'A':[None,'a']})
b = DataFrame({'A':[NaN,'a']})
a.merge(b,on='A', how = 'outer')

,A
0,None
1,a


In [33]:
a = DataFrame({'A':[NaN,'a']})
b = DataFrame({'A':[None,'a']})
a.merge(b,on='A', how = 'outer')

,A
0,NaN
1,a


无论两边都是None,都是NaN，还是都有,相关的列都会被正确的匹配。 注意一边是None,一边是NaN的时候。会以左侧的结果为准。

**注意这和空值在postgresql中的表现不一样， 在postgresql中， join时两边的空值会被判定为不同的数值**

### 在groupby中的表现

In [38]:
d = DataFrame({'A':[1,1,1,1,2],'B':[None,None,'a','a','b']})
d.groupby(['A','B']).apply(len)

A  B
1  a    2
2  b    1
dtype: int64

可以看到(1, NaN)对应的组直接被忽略了

In [39]:
d = DataFrame({'A':[1,1,1,1,2],'B':[None,None,'a','a','b']})
d.groupby(['A','B']).apply(len)

A  B
1  a    2
2  b    1
dtype: int64

(1,None)的组被直接忽略了

In [35]:
d = DataFrame({'A':[1,1,1,1,2],'B':[None,NaN,'a','a','b']})
d.groupby(['A','B']).apply(len)

A  B
1  a    2
2  b    1
dtype: int64

那么上面这个应该没啥意外的

## 写入数据库的影响

往数据库中写入时NaN不可处理，需转换成None,否则会报错。作为pandas老司机， 至少有两种替换方法

In [10]:
s = Series([None,NaN,'a'])
s

0    None
1     NaN
2       a
dtype: object

In [11]:
s.replace([NaN],None)

0    None
1    None
2       a
dtype: object

In [12]:
s[s.isnull()]=None
s

0    None
1    None
2       a
dtype: object

然而你们图森破， 看下面的例子

In [13]:
s = Series([NaN,1])
s

0    NaN
1    1.0
dtype: float64

In [14]:
s.replace([NaN], None)

0    NaN
1    1.0
dtype: float64

In [15]:
s[s.isnull()] = None
s

0    NaN
1    1.0
dtype: float64

Series又一声不吭的自动把None替换成了NaN

这时候可以使用第三种方法处理

In [16]:
s.where(s.notnull(), None)

0    None
1       1
dtype: object

where语句会遍历s中所有的元素，逐一检查条件表达式， 如果成立， 从原来的s取元素； 否则用None填充。 这回没有自动替换成NaN

<table>
</table>

### 在pd.cut中的影响

In [ ]:
pd.cut支持NaN但是不支持

In [30]:
import pandas as pd
pd.cut(Series([NaN]),[1,2])

0    NaN
dtype: category
Categories (1, object): [(1, 2]]

In [29]:
import pandas as pd
pd.cut(Series([None]),[1,2])

TypeError: unorderable types: int() > NoneType()

## None vs NaN要点总结
1. 在pandas中， 如果其他的数据都是数值类型， pandas会把None自动替换成NaN, 甚至能无效化```s[s.isnull()]= None```,和```s.replace(NaN, None)```操作的效果无效化。 这时需要用where函数才能进行替换。 

2. None能够直接被导入数据库作为空值处理， 包含NaN的数据导入时会报错。

3. numpy的很多函数能自动处理NaN，但是如果遇到None就会报错。

4. None和NaN都不能被pandas的groupby函数处理，包含None或者NaN的组都会被忽略。

5. merge是，不管两边都是None,都是NaN，还是一边None，一边NaN，都能恰当的merge

等值性比较的总结:

|   	|   两边都是None	| 两边都是NaN  	|一边None,一边NaN
|---	|---	|---	|
| 等值性判断? | True	| False  	| False
| 存在于tuple中，整个tuple的等值性判断结果?  	|   True	| True  	|False
| 作为np.array中的元素时，等值性判断结果? 	| True  	| False   	|False
| 作为Series的元素时，等值性判断的结果? 	| False  	| False   	|False
| numpy.testing.assert_equals 	| True  	| True   	|False
| Series.equals 	| True  	| True   	|True

由于等值性比较方面，None和NaN在各场景下表现不太一致，为了不给自己惹不必要的麻烦和额外的记忆负担。 实践中，遵循以下两个即可
* 如果不需要使用numpy自动处理缺失值的函数我一般会把两侧都处理成None；否则统一处理成NaN。 
* 如果要判断Series,numpy.array整体的等值性，用专门的Series.equals,numpy.array函数去处理，不要自己用```==```判断